# Highway with SB3's DQN

##  Warming up
We start with a few useful installs and imports:

In [ ]:
# Install environment and agent
!pip install setuptools==65.5.0
!pip install highway-env==1.5
!pip install stable-baselines3[extra]==1.7.0

!pip install tensorboardx gym pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg

# clone specific branch for script/utils to record and show videos
!git clone https://github.com/eleurent/highway-env.git  --branch "v1.6" --single-branch

In [ ]:
# Environment
import gym
import highway_env

# Agent
from stable_baselines3 import DQN

# Visualization utils
%load_ext tensorboard
import sys
from tqdm.notebook import trange
sys.path.insert(0, '/content/highway-env/scripts/')
from utils import record_videos, show_videos

# for suppress warnings
import warnings

## Training
Run tensorboard locally to visualize training.

In [ ]:
%tensorboard --logdir "highway_dqn"

In [ ]:
model = DQN('MlpPolicy', "highway-fast-v0",
            policy_kwargs=dict(net_arch=[256, 256]),
            learning_rate=5e-4,
            buffer_size=15000,
            learning_starts=200,
            batch_size=32,
            gamma=0.8,
            train_freq=1,
            gradient_steps=1,
            target_update_interval=50,
            exploration_fraction=0.7,
            verbose=1,
            tensorboard_log="highway_dqn/")
with warnings.catch_warnings():
    warnings.simplefilter('ignore') # suppress the df append usage warning
    model.learn(int(2e4))



## Testing

Visualize a few episodes

In [ ]:
env = gym.make("highway-fast-v0")
env = record_videos(env)
for episode in trange(3, desc="Test episodes"):
    obs, done = env.reset(), False
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(int(action))
env.close()
show_videos()
